In [139]:
import sys
sys.path.append("../")

import pandas as pd
import plotly.graph_objects as go
from dateutil import parser
import datetime as dt

from api.oanda_api import OandaApi
from infrastructure.instrument_collection import instrumentCollection as ic
from db.db import DataDB

pd.set_option('display.max_rows', None)

In [140]:
database = DataDB()

In [141]:
ic.LoadInstrumentsDB()

In [142]:
pairs = []
for pair, val in ic.instruments_dict.items():
    if val.ins_type == "CURRENCY" and "GBP" in pair:
        pairs.append(pair)

In [143]:
pairs

['GBP_ZAR',
 'GBP_HKD',
 'GBP_CHF',
 'GBP_AUD',
 'GBP_CAD',
 'GBP_USD',
 'GBP_SGD',
 'GBP_PLN',
 'GBP_NZD',
 'GBP_JPY',
 'EUR_GBP']

In [144]:
data = []
api = OandaApi()
for p in pairs:
    d_temp = api.get_candles_df(pair_name=p, granularity="D",count=400)
    d_temp['gain'] = ((d_temp.mid_c - d_temp.mid_o) / d_temp.mid_c) * 100
    d_temp['pair'] = p
    if '_GBP' in p:
        d_temp['gain'] = d_temp['gain'] * -1
    data.append(d_temp)
candles_df = pd.concat(data)
candles_df.reset_index(drop=True, inplace=True)
candles_df.time = candles_df.time.dt.date

In [145]:
candles_df.tail()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,gain,pair
4384,2025-03-27,45060,0.83382,0.83725,0.83156,0.83686,0.83347,0.83687,0.83149,0.83665,0.83418,0.83785,0.83162,0.83707,-0.363263,EUR_GBP
4385,2025-03-30,58661,0.83661,0.83794,0.83498,0.83721,0.83589,0.83787,0.83491,0.83706,0.83733,0.83800,0.83503,0.83736,-0.071667,EUR_GBP
4386,2025-03-31,50571,0.83723,0.83788,0.83502,0.83511,0.83670,0.83782,0.83495,0.83499,0.83776,0.83795,0.83508,0.83523,0.253859,EUR_GBP
4387,2025-04-01,58726,0.83501,0.83878,0.83266,0.83446,0.83467,0.83869,0.83257,0.83419,0.83535,0.83886,0.83273,0.83472,0.065911,EUR_GBP
4388,2025-04-02,124021,0.83443,0.84484,0.83230,0.84354,0.83343,0.84477,0.83220,0.84344,0.83543,0.84492,0.83239,0.84365,-1.079972,EUR_GBP


In [146]:
calendar_data = database.query_all(DataDB.CALENDAR_COLL)

In [147]:
calendar_data_df = pd.DataFrame.from_dict(calendar_data)

In [148]:
calendar_data_df.head()

,date,country,category,event,symbol,actual,previous,forecast
0,2021-05-04,australia,balance of trade,balance of trade,AUITGSB,A$5.574B,A$7.595B,A$7B
1,2021-05-04,canada,balance of trade,balance of trade,CATBTOTB,C$-1.14B,C$1.42B,C$0.5B
2,2021-05-04,united states,balance of trade,balance of trade,USTBTOT,$-74.4B,$-70.5B,$-73.4B
3,2021-05-07,germany,balance of trade,balance of trade,GRTBALE,€20.5B,€18.2B,€17.6B
4,2021-05-07,united states,non farm payrolls,non farm payrolls,NFP TCH,266K,770K,950K


In [149]:
calendar_data_df.date = calendar_data_df.date.dt.date

In [150]:
calendar_data_df_uk = calendar_data_df[calendar_data_df.country == "united kingdom"].copy()

In [151]:
calendar_data_df_uk.head(10)

,date,country,category,event,symbol,actual,previous,forecast
9,2021-05-12,united kingdom,balance of trade,balance of trade,UKTBTTBA,£-2B,£-0.9B,£-6.1B
13,2021-05-18,united kingdom,claimant count change,claimant count change,UNITEDKINCLACOUCHA,-15.1K,-19.4K,25K
16,2021-05-19,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,1.5%,0.7%,1.4%
21,2021-05-20,united kingdom,consumer confidence,gfk consumer confidence,UKCCI,-9,-15,-14
44,2021-06-11,united kingdom,balance of trade,balance of trade,UKTBTTBA,£-0.9B,£-2B,£-3.2B
45,2021-06-15,united kingdom,claimant count change,claimant count change,UNITEDKINCLACOUCHA,-92.6K,-55.8K,-62K
48,2021-06-16,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,2.1%,1.5%,1.8%
55,2021-06-24,united kingdom,consumer confidence,gfk consumer confidence,UKCCI,-9,-9,-7
72,2021-07-09,united kingdom,balance of trade,balance of trade,UKTBTTBA,£0.9B,£-1.6B,£-1.4B
78,2021-07-14,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,2.5%,2.1%,2.3%


In [152]:
calendar_data_df_uk.info()

<class 'pandas.core.frame.DataFrame'>
Index: 65 entries, 9 to 465
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      65 non-null     object
 1   country   65 non-null     object
 2   category  65 non-null     object
 3   event     65 non-null     object
 4   symbol    65 non-null     object
 5   actual    65 non-null     object
 6   previous  65 non-null     object
 7   forecast  65 non-null     object
dtypes: object(8)
memory usage: 4.6+ KB


In [153]:
for col in ['actual', 'previous', 'forecast']:
    for sy in ['£', '%', 'B', 'K']:
        calendar_data_df_uk[col] = calendar_data_df_uk[col].str.replace(sy, "")
    calendar_data_df_uk[col].replace('', 0, inplace=True)
    calendar_data_df_uk[col] = calendar_data_df_uk[col].astype(float)

C:\Users\Concordia\AppData\Local\Temp\ipykernel_6104\681705923.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\Concordia\AppData\Local\Temp\ipykernel_6104\681705923.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpl

In [154]:
date_match = {}
for orig in calendar_data_df_uk.date.unique():
    d = orig
    tries = 0
    while d not in candles_df.time.values:
        d = d + dt.timedelta(days=1)
        tries += 1
        if tries > 1000:
            print("Failed")
            break
    date_match[orig] = d

In [155]:
calendar_data_df_uk['orig_date'] = calendar_data_df_uk.date
calendar_data_df_uk.date = [date_match[x] for x in calendar_data_df_uk.date]

In [156]:
calendar_data_df_uk['delta_prev'] = calendar_data_df_uk.actual - calendar_data_df_uk.previous
calendar_data_df_uk['delta_fc'] = calendar_data_df_uk.actual - calendar_data_df_uk.forecast

In [157]:
calendar_data_df_uk.head(2)

,date,country,category,event,symbol,actual,previous,forecast,orig_date,delta_prev,delta_fc
9,2023-09-18,united kingdom,balance of trade,balance of trade,UKTBTTBA,-2.0,-0.9,-6.1,2021-05-12,-1.1,4.1
13,2023-09-18,united kingdom,claimant count change,claimant count change,UNITEDKINCLACOUCHA,-15.1,-19.4,25.0,2021-05-18,4.3,-40.1


In [158]:
candles_an = candles_df[['time', 'pair', 'gain']].copy()

In [159]:
candles_an.head(2)

,time,pair,gain
0,2023-09-18,GBP_ZAR,-0.381578
1,2023-09-19,GBP_ZAR,-0.745386


In [160]:
merged = pd.merge(left=candles_an, right=calendar_data_df_uk, left_on='time', right_on='date')

In [161]:
merged[merged.category=='inflation rate'].head(2)

,time,pair,gain,date,country,category,event,symbol,actual,previous,forecast,orig_date,delta_prev,delta_fc
2,2023-09-18,GBP_ZAR,-0.381578,2023-09-18,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,1.5,0.7,1.4,2021-05-19,0.8,0.1
6,2023-09-18,GBP_ZAR,-0.381578,2023-09-18,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,2.1,1.5,1.8,2021-06-16,0.6,0.3


In [162]:
merged.category.unique()

array(['balance of trade', 'claimant count change', 'inflation rate',
       'consumer confidence', 'retail sales mom', 'composite pmi',
       'calendar', 'monthly gdp mom', 'monthly gdp yoy', 'interest rate',
       'manufacturing pmi', 'services pmi'], dtype=object)

In [163]:
c = 'consumer confidence'
df_an = merged[merged.category==c]
print(df_an[df_an.delta_fc >= 0].gain.sum())
print(df_an[df_an.delta_fc < 0].gain.sum())

2.498878176544957
2.998653811853948


In [164]:
import plotly.express as px

In [165]:
cat = 'inflation rate'
df_cat = merged[merged.category==cat]
for p in pairs:
    df_plot = df_cat[df_cat.pair == p]
    print(p)
    fig = px.scatter(df_plot, x="gain", y="delta_prev", trendline="ols")
    fig.show()

GBP_ZAR


GBP_HKD


GBP_CHF


GBP_AUD


GBP_CAD


...

GBP_SGD


GBP_PLN


GBP_NZD


GBP_JPY


EUR_GBP
